<font style ="black" color ="#23BAC4" size = "12" > Proyecto  Final -  Parte 1</font>

### **Parte 1.  Spark Core**

- 1. [Carga y preparación de los tres diferentes csv proporcionados](#1)
    - 1.1 [Datos ficticios](#1.1)
    - 1.3 [Datos agua](#1.2)
    - 1.3 [Datos electricidad](#1.3)
- 2. [Estadísticas](#2)
    - 2.1 [Datos agua](#2.1)
    - 2.2 [Datos electricidad](#2.2)
    - 2.3 [Datos ficticios ](#2.3)
            

#### * En esta primera parte se va a llevar a cabo un análisis exploratorio de los 3 csv aportados, en el que se calculan diferentes estadísticas *

<a id="1"></a>
### * 1. Carga y preparación de los tres diferentes csv proporcionados *


In [1]:
val Datos_ficticios = sc.textFile("hdfs:///eoi/Proyecto-Final/Datos_ficticios.csv")

Datos_ficticios = hdfs:///eoi/Proyecto-Final/Datos_ficticios.csv MapPartitionsRDD[44] at textFile at <console>:27


hdfs:///eoi/Proyecto-Final/Datos_ficticios.csv MapPartitionsRDD[44] at textFile at <console>:27

In [2]:
Datos_ficticios.count

30

Se observa como el nº elementos que contiene dicho rdd es 30

In [3]:
val Datos_agua = sc.textFile("hdfs:///eoi/Proyecto-Final/2013agua.csv")

Datos_agua = hdfs:///eoi/Proyecto-Final/2013agua.csv MapPartitionsRDD[46] at textFile at <console>:27


hdfs:///eoi/Proyecto-Final/2013agua.csv MapPartitionsRDD[46] at textFile at <console>:27

In [4]:
Datos_agua.count

1004895

Se observa como el nº elementos que contiene dicho rdd es 1.004.895

In [5]:
val Datos_electricidad = sc.textFile("hdfs:///eoi/Proyecto-Final/2013electricidad.csv")

Datos_electricidad = hdfs:///eoi/Proyecto-Final/2013electricidad.csv MapPartitionsRDD[48] at textFile at <console>:27


hdfs:///eoi/Proyecto-Final/2013electricidad.csv MapPartitionsRDD[48] at textFile at <console>:27

In [6]:
Datos_electricidad.count

998289

Se observa como el nº elementos que contiene dicho rdd es 998.289

<a id="1.1"></a>
### * 1.1 Datos ficticios *

#### * Obtención de la cabecera del RDD de Datos ficticios*

In [7]:
val Datos_ficticios_cabecera = Datos_ficticios.first() 

Datos_ficticios_cabecera = Edificio;Direccion;Provincia;AforoMaximo


Edificio;Direccion;Provincia;AforoMaximo

#### * Se crea un RDD que no contenga la cabecera*

In [8]:
val Datos_ficticios_ok = Datos_ficticios.filter( _ != Datos_ficticios_cabecera)

Datos_ficticios_ok = MapPartitionsRDD[49] at filter at <console>:31


MapPartitionsRDD[49] at filter at <console>:31

In [9]:
Datos_ficticios_ok.count

29

Se observa como el nº elementos que ahora contiene dicho rdd es 29, una vez eliminada la cabecera 

#### * Se crea un RDD que es el que realmente se utilizará durante el resto del análisis, en el que los datos están separados por ; *

In [10]:
val d_ficticios = Datos_ficticios_ok.map(x => x.split(";")).cache

d_ficticios = MapPartitionsRDD[50] at map at <console>:33


MapPartitionsRDD[50] at map at <console>:33

In [11]:
d_ficticios.take(5) 

[[EDIF_1, Direccion upo-1, Sevilla, 18802], [EDIF_1_CAFETERI, Direccion upo-1, Sevilla, 3571], [EDIF_10, Direccion upo-1, Sevilla, 15912], [EDIF_11, Direccion upo-1, Sevilla, 15176], [EDIF_12, Direccion upo-1, Sevilla, 18747]]

Ejemplo para comprobar que esta correcto, mostrando un array con los 3 primeros elementos del rdd 

#### * Se crea un PairRDD donde la clave es el edificio, como identificador del proyecto y el valor es el de todos los campos obtenidos en el comando anterior *

In [12]:
val id_edificio = d_ficticios.keyBy( _ (0))

id_edificio = MapPartitionsRDD[51] at keyBy at <console>:35


MapPartitionsRDD[51] at keyBy at <console>:35

In [13]:
id_edificio.take(5)

[(EDIF_1,[Ljava.lang.String;@29d68474), (EDIF_1_CAFETERI,[Ljava.lang.String;@4084a0ed), (EDIF_10,[Ljava.lang.String;@3361b089), (EDIF_11,[Ljava.lang.String;@4f237ca2), (EDIF_12,[Ljava.lang.String;@52cba416)]

<a id="1.2"></a>
### * 1. 2 Datos agua *

#### * Se crea un RDD en el que los datos están separados por ; *

In [14]:
val dat_agua = Datos_agua.map(x => x.split(";")).cache

dat_agua = MapPartitionsRDD[52] at map at <console>:29


MapPartitionsRDD[52] at map at <console>:29

In [15]:
dat_agua.take(5)

[[AGUA_EDIF_1, 0, 2013, 01, 01, 00, 00], [AGUA_EDIF_1, 0, 2013, 01, 01, 00, 15], [AGUA_EDIF_1, 0, 2013, 01, 01, 00, 30], [AGUA_EDIF_1, 0, 2013, 01, 01, 00, 45], [AGUA_EDIF_1, 0, 2013, 01, 01, 01, 00]]

Se detecta que existen algunas observaciones denominadas AGUA_UPO en lugar de AGUA_EDIF, dado que:

El edificio denominado UPO representa el total de la universidad para cada uno de los consumos en cada momento. Este total incorpora datos de otros edificios o instalaciones no supervisados.

Se decide partir los datos en 2 RDDs diferentes para analizarlos de manera independiente.

In [16]:
val d_agua = dat_agua.filter(_(0)!="AGUA_UPO")

d_agua = MapPartitionsRDD[53] at filter at <console>:31


MapPartitionsRDD[53] at filter at <console>:31

In [17]:
d_agua.count()

970242

In [18]:
val d_agua_UPO = dat_agua.filter(_(0)=="AGUA_UPO")

d_agua_UPO = MapPartitionsRDD[54] at filter at <console>:31


MapPartitionsRDD[54] at filter at <console>:31

In [19]:
d_agua_UPO.count()

34653

<a id="1.3"></a>
### * 1. 3 Datos electricidad *

#### *  Se crea un RDD en el que los datos están separados por ; *

In [20]:
val dat_electricidad = Datos_electricidad.map(x => x.split(";")).cache

dat_electricidad = MapPartitionsRDD[55] at map at <console>:29


MapPartitionsRDD[55] at map at <console>:29

In [21]:
dat_electricidad.take(5)

[[ELEC_EDIF_1, 1.354, 2013, 01, 01, 00, 00], [ELEC_EDIF_1, 1.521, 2013, 01, 01, 00, 15], [ELEC_EDIF_1, 1.239, 2013, 01, 01, 00, 30], [ELEC_EDIF_1, 1.347, 2013, 01, 01, 00, 45], [ELEC_EDIF_1, 1.3, 2013, 01, 01, 01, 00]]

De igual manera como ha ocurrido con el agua, se detecta que existen algunas observaciones denominadas ELEC_UPO en lugar de ELEC_EDIF, en este caso también se decide partir los datos en 2 RDDs diferentes para analizarlos de manera independiente.

In [22]:
val d_electricidad = dat_electricidad.filter(x => (x(0)!="ELEC_UPO"))

d_electricidad = MapPartitionsRDD[56] at filter at <console>:31


MapPartitionsRDD[56] at filter at <console>:31

In [23]:
d_electricidad.count()

963249

In [24]:
val d_electricidad_UPO = dat_electricidad.filter(x => (x(0)=="ELEC_UPO"))

d_electricidad_UPO = MapPartitionsRDD[57] at filter at <console>:31


MapPartitionsRDD[57] at filter at <console>:31

In [25]:
d_electricidad_UPO.count()

35040

<a id="2"></a>
### * 2. Estadísticas*

<a id="2.1"></a>
### *2.1 Agua *

#### *Calculo del valor de la lectura en m3 por cada edificio, se crea un RDD clave(Edificio) -  valor(m3), con valor decimal y se ordenan de mayor a menor*

In [26]:
val agua_por_edificio_1 = d_agua.map(x => (x(0),x(1).toDouble))

agua_por_edificio_1 = MapPartitionsRDD[58] at map at <console>:33


MapPartitionsRDD[58] at map at <console>:33

In [27]:
agua_por_edificio.take(5)

[(AGUA_EDIF_1,0.0), (AGUA_EDIF_1,0.0), (AGUA_EDIF_1,0.0), (AGUA_EDIF_1,0.0), (AGUA_EDIF_1,0.0)]

#### *Se analiza si existen valores negativos y se eliminan, ya que no tienen sentido*

In [28]:
val count_neg = agua_por_edificio.filter(x => x._2 <0).count()

count_neg = 0


0

In [29]:
val agua_por_edificio = agua_por_edificio_1.filter(x => x._2 >= 0)

agua_por_edificio = MapPartitionsRDD[60] at filter at <console>:35


MapPartitionsRDD[60] at filter at <console>:35

In [30]:
agua_por_edificio.count()

970095

In [31]:
val total_agua_por_edificio = agua_por_edificio.reduceByKey((a,b) => a+b).sortBy(x => x._2,false)

total_agua_por_edificio = MapPartitionsRDD[66] at sortBy at <console>:37


MapPartitionsRDD[66] at sortBy at <console>:37

In [32]:
total_agua_por_edificio.take(28)

[(AGUA_EDIF_20,6367.870000000003), (AGUA_EDIF_16,4197.345999999506), (AGUA_EDIF_15,3045.9600000001237), (AGUA_EDIF_7,1807.7299999998027), (AGUA_EDIF_17,1219.489999999943), (AGUA_EDIF_3,952.7199999999352), (AGUA_EDIF_36,925.799999999867), (AGUA_EDIF_1_CAFETERI,709.8699999999867), (AGUA_EDIF_1,709.8699999999565), (AGUA_EDIF_10,628.2499999999555), (AGUA_EDIF_11,527.4899999999997), (AGUA_EDIF_6,479.7899999999386), (AGUA_EDIF_4,472.3299999999401), (AGUA_EDIF_32,401.42999999994436), (AGUA_EDIF_12,396.7199999999699), (AGUA_EDIF_21,387.07199999997704), (AGUA_EDIF_2,369.8799999999802), (AGUA_EDIF_8,348.9499999999779), (AGUA_EDIF_17_COMEDOR,344.9100000000061), (AGUA_EDIF_43,337.2699999999959), (AGUA_EDIF_14,301.39999999999054), (AGUA_EDIF_5,279.1200000000026), (AGUA_EDIF_18_KIOSCO,182.85000000000474), (AGUA_EDIF_44,163.3200000000034), (AGUA_EDIF_13,126.06000000000282), (AGUA_EDIF_31,90.3800000000023), (AGUA_EDIF_42,6.159999999999988), (AGUA_EDIF_45,0.0)]

#### * La media de consumo por edificio es de 920 m3*

In [33]:
val media_agua_edificios = total_agua_por_edificio.values.mean

media_agua_edificios = 920.7156428571006


920.7156428571006

#### *Se calcula el máximo y mínimo*

In [34]:
val max_agua_edificios = total_agua_por_edificio.reduceByKey((a,b) => a+b).sortBy(x => x._2,false).first()

max_agua_edificios = (AGUA_EDIF_20,6367.870000000003)


(AGUA_EDIF_20,6367.870000000003)

In [35]:
val min_agua_edificios = total_agua_por_edificio.reduceByKey((a,b) => a+b).sortBy(x => x._2,true).first()

min_agua_edificios = (AGUA_EDIF_45,0.0)


(AGUA_EDIF_45,0.0)

Es descable el consumo del Edificio número 20 ya que posee casi 7 veces el valor de la media, como es obvio la media esta muy influenciada por los extremos.Además el Edificio 45 no ha tenido consumo de agua, lo cual es raro e intuyo quee será  un edificio destinado a una actividad muy especifica como por ejemplo podría ser almacenaje

#### * Para cuantificar la dispersión de los datos entre edificios se calcula la desviación típica*

In [36]:
val des_agua_edificios = total_agua_por_edificio.values.stdev

des_agua_edificios = 1383.2643465761612


1383.2643465761612

#### * Se aplica reduce para obtener el sumatorio de los consumos de agua en edificios*

In [37]:
val total_agua = agua_por_edificio.values.reduce( (a,b) => a + b)

total_agua = 25780.03799999986


25780.03799999986

#### * Se calcula el sumatorio de los consumos de agua en UPO*

In [47]:
val neg_upo_agua = d_agua_UPO.filter(x => x(1).toDouble < 0 ).count()

neg_upo_agua = 0


0

No posee valores negativos

In [48]:
d_agua_UPO.take(5)

[[AGUA_UPO, 0.62, 2013, 01, 01, 00, 00], [AGUA_UPO, 0.33, 2013, 01, 01, 00, 15], [AGUA_UPO, 0.3, 2013, 01, 01, 00, 30], [AGUA_UPO, 0.25, 2013, 01, 01, 00, 45], [AGUA_UPO, 0.26, 2013, 01, 01, 01, 00]]

In [49]:
val total_agua_upo = d_agua_UPO.map(x => (x(0),x(1).toDouble)).values.reduce((a,b) => a + b)

total_agua_upo = 39388.879999999896


39388.879999999896

#### * Dado que este último RDD incorpora datos de otros edificios o instalaciones no supervisados, la resta entre ambos arroja el consumo de instalaciones no supervisadas*

In [50]:
val total_agua_no_supervisada = total_agua_upo - total_agua

total_agua_no_supervisada = 13608.842000000037


13608.842000000037

Se observa como hay 13.626 m3 de agua consumida por instalaciones no supervisadas

<a id="2.2"></a>
### *2.2 Electricidad*

#### *De igual manera, para la electriciad calculo del valor del kW/h por cada edificio, se crea un RDD clave(Edificio) - valor(kW/h), con valor decimal y se ordenan de mayor a menor*

In [51]:
val elec_por_edificio_1 = d_electricidad.map(x => (x(0),x(1).toDouble))

elec_por_edificio_1 = MapPartitionsRDD[88] at map at <console>:33


MapPartitionsRDD[88] at map at <console>:33

#### *Se analiza si existen valores negativos y se eliminan, ya que no tienen sentido*

In [52]:
val neg_elec = elec_por_edificio_1.filter(x => x._2 <0).count()

neg_elec = 2221


2221

In [53]:
val elec_por_edificio = elec_por_edificio_1.filter(x => x._2 >= 0)

elec_por_edificio = MapPartitionsRDD[90] at filter at <console>:35


MapPartitionsRDD[90] at filter at <console>:35

In [54]:
elec_por_edificio.count()

961028

In [55]:
val total_elec_por_edificio = elec_por_edificio.reduceByKey( (a,b) => a+b).sortBy(x => x._2,false)

MapPartitionsRDD[96] at sortBy at <console>:37

total_elec_por_edificio = MapPartitionsRDD[96] at sortBy at <console>:37


In [56]:
total_elec_por_edificio.take(30)

[(ELEC_EDIF_20,1625734.7639999923), (ELEC_EDIF_21,579830.630000001), (ELEC_EDIF_45,285486.19299999875), (ELEC_EDIF_7,266688.74300000095), (ELEC_EDIF_15,263737.3690000025), (ELEC_EDIF_14,252734.938999999), (ELEC_EDIF_10,224691.37899999702), (ELEC_EDIF_17,219905.69300000192), (ELEC_EDIF_2,218905.90800000125), (ELEC_EDIF_11,195154.23400000075), (ELEC_EDIF_3,173988.7100000009), (ELEC_EDIF_32,160687.17600000033), (ELEC_EDIF_1,143819.20200000048), (ELEC_EDIF_6,135662.6199999995), (ELEC_EDIF_1_CAFETERI,123496.3539999999), (ELEC_EDIF_44,105796.46400000073), (ELEC_EDIF_12,82013.96899999982), (ELEC_EDIF_17_COMEDOR,79210.64399999977), (ELEC_EDIF_31,57470.26199999967), (ELEC_EDIF_8,53756.02600000007), (ELEC_EDIF_5,41959.54400000007), (ELEC_EDIF_4,35481.169000000176), (ELEC_EDIF_18_KIOSCO,30028.251000000007), (ELEC_EDIF_13,27464.337000000156), (ELEC_EDIF_16,23105.5220000001), (ELEC_EDIF_36,9568.368000000246), (ELEC_EDIF_43,7829.521000000384), (ELEC_EDIF_42,6325.0609999988665)]

#### *La media de consumo por edificio es de 193.710,93 kW/h *

In [40]:
val media_elec_edificios = total_elec_por_edificio.values.mean

media_elec_edificios = 193710.93224999987


193710.93224999987

#### *Se calculan los máximos y mínimos*

In [41]:
val max_elec_edificios = total_elec_por_edificio.reduceByKey((a,b) => a+b).sortBy(x => x._2,false).first()

max_elec_edificios = (ELEC_EDIF_20,1625734.7639999923)


(ELEC_EDIF_20,1625734.7639999923)

In [42]:
val min_elec_edificios = total_elec_por_edificio.reduceByKey((a,b) => a+b).sortBy(x => x._2,true).first()

min_elec_edificios = (ELEC_EDIF_42,6325.0609999988665)


(ELEC_EDIF_42,6325.0609999988665)

Se observa como la media es 8 veces menos que el máximo y el mínimo 30 veces menos

#### *Para cuantificar la dispersión de los datos entre edificios se calcula la desviación típica*

In [43]:
val des_elec_edificios = total_elec_por_edificio.values.stdev

des_elec_edificios = 301354.69640557136


301354.69640557136

Se aprecia como los datos de electricidad son muchísimo más dispersos que los datos del agua aunque utilicen diferentes escalas de medición

#### *Se aplica reduce para obtener el sumatorio de los consumos de electricidad en edificios*

In [44]:
val total_elec = elec_por_edificio.values.reduce( (a,b) => a + b)

total_elec = 5423906.103000017


5423906.103000017

#### *Se calcula el sumatorio de los consumos de electricidad en UPO*

In [57]:
val neg_upo_elec = d_electricidad_UPO.filter(x => x(1).toDouble < 0 ).count()

neg_upo_elec = 0


0

No posee negativos

In [45]:
val total_elec_upo = d_electricidad_UPO.map(x => (x(0),x(1).toDouble)).values.reduce((a,b) => a + b)

total_elec_upo = 1.2911443386999983E7


1.2911443386999983E7

#### *Dado que este RDD incorpora datos de otros edificios o instalaciones no supervisados, la resta entre ambos arroja el consumo de instalaciones no supervisadas*

In [46]:
val total_elec_no_supervisada = total_elec_upo - total_elec

total_elec_no_supervisada = 7487537.283999966


7487537.283999966

Se observa como hay 7.487.537.28 kW/h de electricidad consumida por instalaciones no supervisadas

<a id="2.3"></a>
### *2. 3 Datos ficticios*

Se procede a relizar la unión de las bases de datos de agua y electricidad con la base de datos fictios para analizar los consumos en función de las direcciones,estos datos se uniran a través del id Edificio.

Para unirlas el identificador no es igual ya que el csv de agua contiene AGUA_EDIF_X por lo que hay que deshacerse de la parte AGUA_ del csv original llamado Datos_agua, de igual manera ocurre con la electricidad.

También es remarcable que este cambio habría sido más eficiente realizarlo al comienzo del análisis para no escribir el mismo código otra vez y ahorrar pasos e ir mucho más rápido.

(Quería reflejar la realidad del análisis llevado a cabo, en un proyecto primero hay que marcarse los objetivos a conseguir y después analizar el cómo, si desde un inicio hubiera tenido claro el objetivo de unir las bases de datos para esta primera parte me habria ahorrado gran parte de los pasos siguientes). 

<a id="2.3.1"></a>
### *2. 3. 1 Datos ficticios union con agua*

In [47]:
val agua_sin_ = Datos_agua.map(x => x.replace( "AGUA_", "" ))

agua_sin_ = MapPartitionsRDD[67] at map at <console>:29


MapPartitionsRDD[67] at map at <console>:29

In [48]:
val agua_edif_sin = agua_sin_.map(x => x.split(";")).cache

agua_edif_sin = MapPartitionsRDD[68] at map at <console>:31


MapPartitionsRDD[68] at map at <console>:31

In [49]:
agua_edif_sin.take(10)

[[EDIF_1, 0, 2013, 01, 01, 00, 00], [EDIF_1, 0, 2013, 01, 01, 00, 15], [EDIF_1, 0, 2013, 01, 01, 00, 30], [EDIF_1, 0, 2013, 01, 01, 00, 45], [EDIF_1, 0, 2013, 01, 01, 01, 00], [EDIF_1, 0, 2013, 01, 01, 01, 15], [EDIF_1, 0, 2013, 01, 01, 01, 30], [EDIF_1, 0, 2013, 01, 01, 01, 45], [EDIF_1, 0, 2013, 01, 01, 02, 00], [EDIF_1, 0, 2013, 01, 01, 02, 15]]

In [50]:
val agua_edif_sin_N_UPO = agua_edif_sin.filter(x=>(x(0)!="UPO"))

agua_edif_sin_N_UPO = MapPartitionsRDD[69] at filter at <console>:33


MapPartitionsRDD[69] at filter at <console>:33

In [51]:
val agua_por_edifi_SIN = agua_edif_sin_N_UPO.map(x => (x(0),x(1).toDouble))

agua_por_edifi_SIN = MapPartitionsRDD[70] at map at <console>:35


MapPartitionsRDD[70] at map at <console>:35

In [52]:
val total_agua_por_edif_SIN = agua_por_edifi_SIN.reduceByKey((v1,v2)=> v1+v2).sortBy(_._2,false)

total_agua_por_edif_SIN = MapPartitionsRDD[76] at sortBy at <console>:37


MapPartitionsRDD[76] at sortBy at <console>:37

In [53]:
total_agua_por_edif_SIN.take(28)

[(EDIF_20,6367.870000000003), (EDIF_16,4179.874999999503), (EDIF_15,3045.9600000001237), (EDIF_7,1807.7299999998027), (EDIF_17,1219.489999999943), (EDIF_3,952.7199999999352), (EDIF_36,925.799999999867), (EDIF_1_CAFETERI,709.8699999999867), (EDIF_1,709.8699999999565), (EDIF_10,628.2499999999555), (EDIF_11,527.4899999999997), (EDIF_6,479.7899999999386), (EDIF_4,472.3299999999401), (EDIF_32,401.42999999994436), (EDIF_12,396.7199999999699), (EDIF_21,387.07199999997704), (EDIF_2,369.8799999999802), (EDIF_8,348.9499999999779), (EDIF_17_COMEDOR,344.9100000000061), (EDIF_43,337.2699999999959), (EDIF_14,301.39999999999054), (EDIF_5,279.1200000000026), (EDIF_18_KIOSCO,182.85000000000474), (EDIF_44,163.3200000000034), (EDIF_13,126.06000000000282), (EDIF_31,90.3800000000023), (EDIF_42,6.159999999999988), (EDIF_45,0.0)]

In [54]:
val agua_ficticios = d_ficticios.map(x => (x(0), x(1)))

agua_ficticios = MapPartitionsRDD[77] at map at <console>:35


MapPartitionsRDD[77] at map at <console>:35

In [55]:
agua_ficticios.take(28)

[(EDIF_1,Direccion upo-1), (EDIF_1_CAFETERI,Direccion upo-1), (EDIF_10,Direccion upo-1), (EDIF_11,Direccion upo-1), (EDIF_12,Direccion upo-1), (EDIF_13,Direccion upo-1), (EDIF_14,Direccion upo-1), (EDIF_15,Direccion upo-1), (EDIF_16,Direccion upo-1), (EDIF_17,Direccion upo-1), (EDIF_17_COMEDOR,Direccion upo-1), (EDIF_18_KIOSCO,Direccion upo-1), (EDIF_2,Direccion upo-2), (EDIF_20,Direccion upo-2), (EDIF_21,Direccion upo-2), (EDIF_3,Direccion upo-3), (EDIF_31,Direccion upo-3), (EDIF_32,Direccion upo-3), (EDIF_36,Direccion upo-3), (EDIF_4,Direccion upo-4), (EDIF_42,Direccion upo-4), (EDIF_43,Direccion upo-4), (EDIF_44,Direccion upo-4), (EDIF_45,Direccion upo-4), (EDIF_5,Direccion upo-5), (EDIF_6,Direccion upo-6), (EDIF_7,Direccion upo-7), (EDIF_8,Direccion upo-8)]

In [56]:
val union_agua_ficticios =  total_agua_por_edif_SIN.join(agua_ficticios)

union_agua_ficticios = MapPartitionsRDD[80] at join at <console>:49


MapPartitionsRDD[80] at join at <console>:49

In [57]:
union_agua_ficticios.take(5)

[(EDIF_17_COMEDOR,(344.9100000000061,Direccion upo-1)), (EDIF_18_KIOSCO,(182.85000000000474,Direccion upo-1)), (EDIF_21,(387.07199999997704,Direccion upo-2)), (EDIF_2,(369.8799999999802,Direccion upo-2)), (EDIF_3,(952.7199999999352,Direccion upo-3))]

In [58]:
val agua_direccion = union_agua_ficticios.map(x=> (( x._2 ).toString.split( "," )))

agua_direccion = MapPartitionsRDD[81] at map at <console>:51


MapPartitionsRDD[81] at map at <console>:51

In [59]:
agua_direccion.take(28)

[[(344.9100000000061, Direccion upo-1)], [(182.85000000000474, Direccion upo-1)], [(387.07199999997704, Direccion upo-2)], [(369.8799999999802, Direccion upo-2)], [(952.7199999999352, Direccion upo-3)], [(925.799999999867, Direccion upo-3)], [(628.2499999999555, Direccion upo-1)], [(472.3299999999401, Direccion upo-4)], [(527.4899999999997, Direccion upo-1)], [(279.1200000000026, Direccion upo-5)], [(479.7899999999386, Direccion upo-6)], [(396.7199999999699, Direccion upo-1)], [(126.06000000000282, Direccion upo-1)], [(1807.7299999998027, Direccion upo-7)], [(301.39999999999054, Direccion upo-1)], [(348.9499999999779, Direccion upo-8)], [(3045.9600000001237, Direccion upo-1)], [(4179.874999999503, Direccion upo-1)], [(1219.489999999943, Direccion upo-1)], [(709.8699999999867, Direccion upo-1)], [(6.159999999999988, Direccion upo-4)], [(337.2699999999959, Direccion upo-4)], [(90.3800000000023, Direccion upo-3)], [(163.3200000000034, Direccion upo-4)], [(401.42999999994436, Direccion upo

Se aprecia como devuelve los datos  metidos entre [( )], los cuales hay que eliminarlos para agrupar por direccion 

Además, se cambia la posición para tener como id la dirección y el consumo se pasa a decimal

In [60]:
val agua_direc = agua_direccion.map(x => ( x(1).replace( ")" , "" ) , x(0).replace( "(" , "" ).toDouble))

agua_direc = MapPartitionsRDD[82] at map at <console>:53


MapPartitionsRDD[82] at map at <console>:53

In [61]:
agua_direc.take(5)

[(Direccion upo-1,344.9100000000061), (Direccion upo-1,182.85000000000474), (Direccion upo-2,387.07199999997704), (Direccion upo-2,369.8799999999802), (Direccion upo-3,952.7199999999352)]

In [62]:
agua_direc.sortBy(_._2,false)

MapPartitionsRDD[87] at sortBy at <console>:56

In [63]:
agua_direc.take(28)

[(Direccion upo-1,344.9100000000061), (Direccion upo-1,182.85000000000474), (Direccion upo-2,387.07199999997704), (Direccion upo-2,369.8799999999802), (Direccion upo-3,952.7199999999352), (Direccion upo-3,925.799999999867), (Direccion upo-1,628.2499999999555), (Direccion upo-4,472.3299999999401), (Direccion upo-1,527.4899999999997), (Direccion upo-5,279.1200000000026), (Direccion upo-6,479.7899999999386), (Direccion upo-1,396.7199999999699), (Direccion upo-1,126.06000000000282), (Direccion upo-7,1807.7299999998027), (Direccion upo-1,301.39999999999054), (Direccion upo-8,348.9499999999779), (Direccion upo-1,3045.9600000001237), (Direccion upo-1,4179.874999999503), (Direccion upo-1,1219.489999999943), (Direccion upo-1,709.8699999999867), (Direccion upo-4,6.159999999999988), (Direccion upo-4,337.2699999999959), (Direccion upo-3,90.3800000000023), (Direccion upo-4,163.3200000000034), (Direccion upo-3,401.42999999994436), (Direccion upo-4,0.0), (Direccion upo-2,6367.870000000003), (Direccio

Finalmente para obtener el sumatorio por clave se aplica una reduccion por clave, que ahora sí es la direccion 

In [64]:
val sumatorio = agua_direc.reduceByKey((v1,v2)=> v1+v2).sortBy(_._2,false)

sumatorio = MapPartitionsRDD[93] at sortBy at <console>:55


MapPartitionsRDD[93] at sortBy at <console>:55

In [65]:
sumatorio.take(28)

[(Direccion upo-1,12372.744999999442), (Direccion upo-2,7124.82199999996), (Direccion upo-3,2370.329999999749), (Direccion upo-7,1807.7299999998027), (Direccion upo-4,979.0799999999394), (Direccion upo-6,479.7899999999386), (Direccion upo-8,348.9499999999779), (Direccion upo-5,279.1200000000026)]

De igual manera se podría llevar a cabo el análisis de diferentes estadísticas por dirección, pero al ser solamente 8 direcciones y además intentar no ser redundante, decido que no tiene relevancia calcular un máximo o mínimo en este caso

<a id="2.3.1"></a>
### *2. 3. 1 Datos ficticios union con electricidad*

In [66]:
val elec_sin_ = Datos_electricidad.map(x => x.replace( "ELEC_", "" ))

elec_sin_ = MapPartitionsRDD[94] at map at <console>:29


MapPartitionsRDD[94] at map at <console>:29

In [67]:
val elec_edif_sin = elec_sin_.map(x => x.split(";")).cache

elec_edif_sin = MapPartitionsRDD[95] at map at <console>:31


MapPartitionsRDD[95] at map at <console>:31

In [68]:
elec_edif_sin.take(10)

[[EDIF_1, 1.354, 2013, 01, 01, 00, 00], [EDIF_1, 1.521, 2013, 01, 01, 00, 15], [EDIF_1, 1.239, 2013, 01, 01, 00, 30], [EDIF_1, 1.347, 2013, 01, 01, 00, 45], [EDIF_1, 1.3, 2013, 01, 01, 01, 00], [EDIF_1, 1.695, 2013, 01, 01, 01, 15], [EDIF_1, 1.577, 2013, 01, 01, 01, 30], [EDIF_1, 1.288, 2013, 01, 01, 01, 45], [EDIF_1, 1.607, 2013, 01, 01, 02, 00], [EDIF_1, 1.448, 2013, 01, 01, 02, 15]]

In [69]:
val elec_edif_sin_N_UPO = elec_edif_sin.filter(x=>(x(0)!="UPO"))

elec_edif_sin_N_UPO = MapPartitionsRDD[96] at filter at <console>:33


MapPartitionsRDD[96] at filter at <console>:33

In [70]:
val elec_por_edifi_SIN = elec_edif_sin_N_UPO.map(x => (x(0),x(1).toDouble))

elec_por_edifi_SIN = MapPartitionsRDD[97] at map at <console>:35


MapPartitionsRDD[97] at map at <console>:35

In [71]:
val total_elec_por_edif_SIN = elec_por_edifi_SIN.reduceByKey((v1,v2)=> v1+v2).sortBy(_._2,false)

total_elec_por_edif_SIN = MapPartitionsRDD[103] at sortBy at <console>:37


MapPartitionsRDD[103] at sortBy at <console>:37

In [72]:
total_elec_por_edif_SIN.take(28)

[(EDIF_20,1625734.7639999923), (EDIF_21,579830.630000001), (EDIF_45,285486.19299999875), (EDIF_7,266688.74300000095), (EDIF_15,263737.3690000025), (EDIF_14,252734.938999999), (EDIF_10,224691.37899999702), (EDIF_2,218905.90800000125), (EDIF_17,213318.15800000192), (EDIF_11,195154.23400000075), (EDIF_3,173988.7100000009), (EDIF_32,160687.17600000033), (EDIF_1,143819.20200000048), (EDIF_6,135623.20599999957), (EDIF_1_CAFETERI,123496.3539999999), (EDIF_44,105796.46400000073), (EDIF_12,82013.96899999982), (EDIF_17_COMEDOR,79210.64399999977), (EDIF_31,57470.26199999967), (EDIF_8,53756.02600000007), (EDIF_5,41959.54400000007), (EDIF_4,35481.169000000176), (EDIF_18_KIOSCO,30028.251000000007), (EDIF_13,27464.337000000156), (EDIF_16,23105.5220000001), (EDIF_36,9568.368000000246), (EDIF_43,7829.521000000384), (EDIF_42,6325.0609999988665)]

In [73]:
val elec_ficticios = d_ficticios.map(x => (x(0), x(1)))

elec_ficticios = MapPartitionsRDD[104] at map at <console>:35


MapPartitionsRDD[104] at map at <console>:35

In [74]:
elec_ficticios.take(3)

[(EDIF_1,Direccion upo-1), (EDIF_1_CAFETERI,Direccion upo-1), (EDIF_10,Direccion upo-1)]

In [75]:
val union_elec_ficticios =  total_elec_por_edif_SIN.join(agua_ficticios)

union_elec_ficticios = MapPartitionsRDD[107] at join at <console>:49


MapPartitionsRDD[107] at join at <console>:49

In [76]:
union_elec_ficticios.take(5)

[(EDIF_45,(285486.19299999875,Direccion upo-4)), (EDIF_31,(57470.26199999967,Direccion upo-3)), (EDIF_18_KIOSCO,(30028.251000000007,Direccion upo-1)), (EDIF_32,(160687.17600000033,Direccion upo-3)), (EDIF_17_COMEDOR,(79210.64399999977,Direccion upo-1))]

In [77]:
val elec_direccion = union_elec_ficticios.map(x=> (( x._2 ).toString.split( "," )))

elec_direccion = MapPartitionsRDD[108] at map at <console>:51


MapPartitionsRDD[108] at map at <console>:51

In [78]:
elec_direccion.take(5)

[[(285486.19299999875, Direccion upo-4)], [(57470.26199999967, Direccion upo-3)], [(30028.251000000007, Direccion upo-1)], [(160687.17600000033, Direccion upo-3)], [(79210.64399999977, Direccion upo-1)]]

Se lleva a cabo el mismo proceso que antes, se aprecia como devuelve los datos metidos entre [( )], los cuales hay que eliminarlos para agrupar por direccion. Además, se cambia la posición para tener como id la dirección y el consumo se pasa a decimal

In [79]:
val elec_direc = elec_direccion.map(x => ( x(1).replace( ")" , "" ) , x(0).replace( "(" , "" ).toDouble))

elec_direc = MapPartitionsRDD[109] at map at <console>:53


MapPartitionsRDD[109] at map at <console>:53

In [80]:
elec_direc.sortBy(_._2,false)

MapPartitionsRDD[114] at sortBy at <console>:56

In [81]:
elec_direc.take(28)

[(Direccion upo-4,285486.19299999875), (Direccion upo-3,57470.26199999967), (Direccion upo-1,30028.251000000007), (Direccion upo-3,160687.17600000033), (Direccion upo-1,79210.64399999977), (Direccion upo-2,1625734.7639999923), (Direccion upo-3,9568.368000000246), (Direccion upo-2,579830.630000001), (Direccion upo-1,224691.37899999702), (Direccion upo-1,143819.20200000048), (Direccion upo-1,195154.23400000075), (Direccion upo-2,218905.90800000125), (Direccion upo-3,173988.7100000009), (Direccion upo-1,82013.96899999982), (Direccion upo-1,27464.337000000156), (Direccion upo-4,35481.169000000176), (Direccion upo-1,252734.938999999), (Direccion upo-5,41959.54400000007), (Direccion upo-1,263737.3690000025), (Direccion upo-6,135623.20599999957), (Direccion upo-4,6325.0609999988665), (Direccion upo-7,266688.74300000095), (Direccion upo-1,23105.5220000001), (Direccion upo-1,123496.3539999999), (Direccion upo-4,7829.521000000384), (Direccion upo-1,213318.15800000192), (Direccion upo-8,53756.026

In [82]:
val sumatorio2 = elec_direc.reduceByKey((v1,v2)=> v1+v2).sortBy(_._2,false)

sumatorio2 = MapPartitionsRDD[120] at sortBy at <console>:55


MapPartitionsRDD[120] at sortBy at <console>:55

In [83]:
sumatorio2.take(28)

[(Direccion upo-2,2424471.3019999946), (Direccion upo-1,1658774.3580000014), (Direccion upo-4,440918.4079999989), (Direccion upo-3,401714.5160000011), (Direccion upo-7,266688.74300000095), (Direccion upo-6,135623.20599999957), (Direccion upo-8,53756.02600000007), (Direccion upo-5,41959.54400000007)]

De igual manera se podría llevar a cabo el análisis de diferentes estadísticas por dirección, pero al ser solamente 8 direcciones y además intentar no ser redundante, decido que no tiene relevancia calcular un máximo o mínimo en este caso